<a href="https://colab.research.google.com/github/22b1074/Vehicle-Detection-IE-643-/blob/main/Foggy_Data_Collection_IE643_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount the drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

# Upload an image
uploaded = files.upload()


Saving foggy_refined.txt to foggy_refined (1).txt


In [ ]:
zip_file_path = '/content/drive/MyDrive/nozip_images_extracted.zip'  #before saved only images which are in refined_foggy.txt in cityscapes foggy simulated datset

In [ ]:
target_directory = '/content/extracted_images'

In [ ]:
import os
import zipfile
os.makedirs(target_directory, exist_ok=True)

# Open the zip file and extract with new names
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        # Get the original filename
        original_filename = file_info.filename

        # Check if the file matches the pattern for renaming
        if 'leftImg8bit_foggy' in original_filename:
            # Create the new filename by removing 'leftImg8bit_' and keeping the rest
            new_filename = original_filename.replace('leftImg8bit_', '')  # Remove 'leftImg8bit_'


            # Construct the full target path
            target_file_path = os.path.join(target_directory, new_filename)

            # Extract the file with the new name
            with zip_ref.open(file_info) as source_file:
                with open(target_file_path, 'wb') as target_file:
                    target_file.write(source_file.read())

            print(f'Extracted and renamed: {new_filename}')
        else:
            print(f'Skipped: {original_filename} (not a foggy image)')

print(f'Files extracted to: {target_directory}')

Extracted and renamed: aachen_000065_000019_foggy_beta_0.01.png
Extracted and renamed: erfurt_000092_000019_foggy_beta_0.01.png
Extracted and renamed: bremen_000203_000019_foggy_beta_0.01.png
Extracted and renamed: erfurt_000105_000019_foggy_beta_0.01.png
Extracted and renamed: dusseldorf_000027_000019_foggy_beta_0.01.png
Extracted and renamed: dusseldorf_000025_000019_foggy_beta_0.01.png
Extracted and renamed: weimar_000003_000019_foggy_beta_0.01.png
Extracted and renamed: dusseldorf_000144_000019_foggy_beta_0.01.png
Extracted and renamed: bremen_000264_000019_foggy_beta_0.01.png
Extracted and renamed: erfurt_000108_000019_foggy_beta_0.01.png
Extracted and renamed: aachen_000132_000019_foggy_beta_0.01.png
Extracted and renamed: erfurt_000063_000019_foggy_beta_0.01.png
Extracted and renamed: erfurt_000010_000019_foggy_beta_0.01.png
Extracted and renamed: erfurt_000032_000019_foggy_beta_0.01.png
Extracted and renamed: aachen_000091_000019_foggy_beta_0.01.png
Extracted and renamed: erfur

In [ ]:
!kaggle datasets download -d ananthudupa2002/cityscapes-gtfine-trainvaltest


Dataset URL: https://www.kaggle.com/datasets/ananthudupa2002/cityscapes-gtfine-trainvaltest
License(s): DbCL-1.0
100% 239M/240M [00:03<00:00, 90.3MB/s]
100% 240M/240M [00:03<00:00, 77.1MB/s]


In [ ]:
!mkdir -p /content/input/gtfine_trainvaltest
!unzip /content/cityscapes-gtfine-trainvaltest.zip -d /content/input/gtfine_trainvaltest

Streaming output truncated to the last 5000 lines.
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_instanceIds.png  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_labelIds.png  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_polygons.json  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_color.png  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_instanceIds.png  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_labelIds.png  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_polygons.json  
  inflating: /content/input/gtfine_trainvaltest/gtFine/train/strasbourg/strasbourg_000001_051877_gtFine_color.p

In [ ]:
import json
from pathlib import Path

# Define the classes of interest with their corresponding class IDs
classes_of_interest = {
    'bicycle': 0,
    'motorcycle': 1,
    'car': 2,
    'truck': 3,
    'bus': 4
}

def convert_cityscapes_to_yolo(gtfine_folder, output_folder):
    # Ensure the output folder exists
    Path(output_folder).mkdir(parents=True, exist_ok=True)

    # Iterate through each category folder (train, val, test)
    for split in ['train', 'val']:
        split_folder = os.path.join(gtfine_folder, split)
        print(split_folder)

        # Iterate over each city folder in the current split
        for city_folder in os.listdir(split_folder):
            city_path = os.path.join(split_folder, city_folder)

            # Check if the current folder is actually a directory
            if os.path.isdir(city_path):
                # Iterate over each JSON file in the current city folder
                for json_file in os.listdir(city_path):
                    if json_file.endswith('.json'):
                        #print(f"Processing {json_file} in {city_path}")
                        # Load the JSON file
                        with open(os.path.join(city_path, json_file), 'r') as f:
                            data = json.load(f)

                        # Construct the corresponding image name
                        #image_name_base = json_file.replace('_gtFine_polygons.json', '')
                        #image_name = f"{image_name_base}_leftImg8bit.png"  # Assuming images are in .png format

                        # Search for the image in the corresponding city folder in leftImg8bit
                        #image_folder_path = os.path.join(leftimg8bit_folder, split, city_folder)
                        #image_path = os.path.join(image_folder_path, image_name)

                        #if os.path.isfile(json_file):

                            #print(f"Image found: {image_path}")
                            # Prepare to collect bounding boxes
                            bounding_boxes = []

                            # Iterate over objects in the JSON data
                            for obj in data['objects']:
                                category_name = obj['label']
                                if category_name in classes_of_interest:
                                    #print(f"Found object: {category_name}")

                                    # Get the polygon vertices
                                    polygon = obj['polygon']

                                    # Calculate bounding box
                                    x_coords = [point[0] for point in polygon]
                                    y_coords = [point[1] for point in polygon]

                                    xmin = min(x_coords)
                                    xmax = max(x_coords)
                                    ymin = min(y_coords)
                                    ymax = max(y_coords)

                                    # Calculate center coordinates
                                    x_center = (xmin + xmax) / 2
                                    y_center = (ymin + ymax) / 2

                                    # Calculate width and height
                                    width = xmax - xmin
                                    height = ymax - ymin

                                    # Normalize values
                                    image_width = data['imgWidth']
                                    image_height = data['imgHeight']
                                    x_center_normalized = x_center / image_width
                                    y_center_normalized = y_center / image_height
                                    width_normalized = width / image_width
                                    height_normalized = height / image_height

                                    # Append the formatted bounding box (class_id, x_center, y_center, width, height)
                                    bounding_boxes.append(f"{classes_of_interest[category_name]} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}")

                            #print(f"Bounding boxes found: {len(bounding_boxes)}")

                            # Save the bounding boxes to a txt file (same name as json)
                            if bounding_boxes:
                                output_file = os.path.splitext(json_file)[0] + '.txt'
                                with open(os.path.join(output_folder, output_file), 'w') as out_f:
                                    out_f.write("\n".join(bounding_boxes))
                                #print(f"Saved bounding boxes to {output_file}")
                            else:
                                print(f"No bounding boxes found for {json_file}")

# Example usage
gtfine_folder = '/content/input/gtfine_trainvaltest/gtFine'  # Change to your gtFine folder path
#leftimg8bit_folder = '/content/input/cityscapes-leftimg8bit-trainvaltest/leftImg8bit'  # Change to your leftImg8bit folder path
output_folder = '/content/working/labels_final'  # Change to your desired output folder
convert_cityscapes_to_yolo(gtfine_folder, output_folder)


/content/input/gtfine_trainvaltest/gtFine/train
No bounding boxes found for cologne_000108_000019_gtFine_polygons.json
No bounding boxes found for strasbourg_000000_007727_gtFine_polygons.json
No bounding boxes found for strasbourg_000000_016311_gtFine_polygons.json
No bounding boxes found for strasbourg_000000_018153_gtFine_polygons.json
No bounding boxes found for strasbourg_000001_061384_gtFine_polygons.json
No bounding boxes found for strasbourg_000001_016253_gtFine_polygons.json
No bounding boxes found for strasbourg_000001_007524_gtFine_polygons.json
No bounding boxes found for strasbourg_000000_012934_gtFine_polygons.json
No bounding boxes found for strasbourg_000001_004106_gtFine_polygons.json
No bounding boxes found for strasbourg_000000_016247_gtFine_polygons.json
No bounding boxes found for strasbourg_000000_035571_gtFine_polygons.json
No bounding boxes found for strasbourg_000001_001901_gtFine_polygons.json
No bounding boxes found for strasbourg_000001_003159_gtFine_polygon

In [ ]:
!ls /content/working/labels_final

In [ ]:
import shutil
validation_cities = ['munster', 'lindau', 'frankfurt']
#images_input_path = '/content/input/cityscapes-leftimg8bit-trainvaltest/leftImg8bit/'
labels_path = '/content/working/labels_final'
output_labels_train_path = '/content/working/train/labels'
output_labels_val_path = '/content/working/val/labels'
#output_images_train_path = '/content/working/train/images'  # Output path for training images
#output_images_val_path = '/content/working/val/images'

# Create output directories for labels and images
Path(output_labels_train_path).mkdir(parents=True, exist_ok=True)
Path(output_labels_val_path).mkdir(parents=True, exist_ok=True)
#Path(output_images_train_path).mkdir(parents=True, exist_ok=True)
#Path(output_images_val_path).mkdir(parents=True, exist_ok=True)

# Iterate over each annotation file in the labels directory
for label_file in os.listdir(labels_path):
    if label_file.endswith('_gtFine_polygons.txt'):
        # Extract the base name and city information
        base_name = label_file.split('_gtFine_polygons.txt')[0]  # This is 'A'
        city_name = base_name.split('_')[0]  # Assuming 'A' is in the format 'A_a1_a2_a3'

        # Determine if this file is for training or validation
        if city_name.lower() in validation_cities:
            # This is a validation file
            destination_label_path = output_labels_val_path
            #destination_image_path = output_images_val_path
            #images_input_path = os.path.join(images_input_path, 'val')
        else:
            # This is a training file
            destination_label_path = output_labels_train_path
            #destination_image_path = output_images_train_path
            #images_input_path = os.path.join(images_input_path, 'train')

        # Move the label file
        shutil.copy(os.path.join(labels_path, label_file), destination_label_path)
        #print(f"Moved label {label_file} to {destination_label_path}.")

        # Construct the corresponding image file name
        #image_name = f"{base_name}"
        #image_folder_path = os.path.join(images_input_path, base_name.split('_')[0])  # City folder for images
        #image_path = os.path.join(image_folder_path, image_name)

        # Copy the corresponding image file if it exists
        #if os.path.isfile(image_path):
         #   shutil.copy(image_path, destination_image_path)
            #print(f"Copied image {image_name} to {destination_image_path}.")
        #else:
         #   print(f"Image {image_path} not found for label {label_file}.")
        #images_input_path = '/content/input/cityscapes-leftimg8bit-trainvaltest/leftImg8bit/'


In [ ]:
import os
labels_dir = '/content/working/train/labels' #replace train by val also

# Loop through each label file in the directory
for label_file in os.listdir(labels_dir):
    if label_file.endswith('_gtFine_polygons.txt'):
        # Create the new label file name by removing '_gtFine_polygons'
        new_label_file = label_file.replace('_gtFine_polygons', '')  # Result will be 'A_B_C.txt'
        # Rename the file
        os.rename(os.path.join(labels_dir, label_file), os.path.join(labels_dir, new_label_file))
        #print(f'Renamed: {label_file} to {new_label_file}')


In [ ]:
import os
import shutil

# Paths to your files
foggy_text_file_path = '/content/foggy_refined.txt'  # Adjust as necessary
labels_train_directory = '/content/working/train/labels/'  # Training labels directory
labels_val_directory = '/content/working/val/labels/'  # Validation labels directory
target_labels_directory = '/content/extracted_labels_val/'  # Directory to store extracted labels

# Create the target directory for labels
os.makedirs(target_labels_directory, exist_ok=True)

# Read the filenames from the foggy text file
with open(foggy_text_file_path, 'r') as file:
    foggy_filenames = [line.strip() for line in file.readlines()]

# Process each foggy filename to find corresponding label files
for foggy_filename in foggy_filenames:
    # Get the base filename without path
    base_filename = os.path.basename(foggy_filename) # Extract base filename
    # change following accordingly use if related to train for train and val val
    # Determine the new label filename based on foggy filename
    #if foggy_filename.startswith('train'):
     #   label_filename = f'{base_filename}_foggy_beta_0.01.txt'
      #  labels_directory = labels_train_directory
    if foggy_filename.startswith('val'):
        label_filename = f'{base_filename}_foggy_beta_0.02.txt'
        labels_directory = labels_val_directory

    # Full path for the corresponding label file
    full_label_path = os.path.join(labels_directory, f'{base_filename}.txt')

    # Check if the label file exists and copy it to the target directory
    if os.path.exists(full_label_path):
        shutil.copy(full_label_path, os.path.join(target_labels_directory, label_filename))
        print(f'Copied label file: {label_filename}')
    else:
        print(f'Label file not found: {full_label_path}')


Label file not found: /content/working/val/labels/aachen_000004_000019.txt
Label file not found: /content/working/val/labels/aachen_000005_000019.txt
Label file not found: /content/working/val/labels/aachen_000007_000019.txt
Label file not found: /content/working/val/labels/aachen_000008_000019.txt
Label file not found: /content/working/val/labels/aachen_000009_000019.txt
Label file not found: /content/working/val/labels/aachen_000010_000019.txt
Label file not found: /content/working/val/labels/aachen_000011_000019.txt
Label file not found: /content/working/val/labels/aachen_000012_000019.txt
Label file not found: /content/working/val/labels/aachen_000015_000019.txt
Label file not found: /content/working/val/labels/aachen_000016_000019.txt
Label file not found: /content/working/val/labels/aachen_000029_000019.txt
Label file not found: /content/working/val/labels/aachen_000030_000019.txt
Label file not found: /content/working/val/labels/aachen_000031_000019.txt
Label file not found: /co

In [ ]:
/content/extracted_labels_train/aachen_000004_000019_foggy_beta_0.01.txt

In [ ]:
/content/extracted_images/aachen_000004_000019_foggy_beta_0.01.png

In [ ]:
# This cell is same as previous cell except for images so did small chanegs and used previous cell
import os
import shutil


extracted_labels_directory = '/content/extracted_labels_val'  # Source directory for label files
extracted_images_directory = '/content/extracted_images'  # Source directory for image files
target_labels_directory = '/content/drive/MyDrive/foggy_IE643/val/labels'  # Target directory for labels
target_images_directory = '/content/drive/MyDrive/foggy_IE643/val/images'  # Target directory for images

# Create target directories if they don't exist
os.makedirs(target_labels_directory, exist_ok=True)
os.makedirs(target_images_directory, exist_ok=True)

# Loop through each label file in the extracted labels directory
for label_file in os.listdir(extracted_labels_directory):
    # Check if the file is a .txt file
    if label_file.endswith('.txt'):
        # Get the base filename without extension
        base_filename = os.path.splitext(label_file)[0]

        # Define corresponding image filename
        image_file = f'{base_filename}.png'

        # Source paths
        label_source_path = os.path.join(extracted_labels_directory, label_file)
        image_source_path = os.path.join(extracted_images_directory, image_file)

        # Copy label file to target labels directory
        shutil.copy(label_source_path, os.path.join(target_labels_directory, label_file))
        print(f'Copied label file: {label_file}')

        # Check if the corresponding image file exists and copy it
        if os.path.exists(image_source_path):
            shutil.copy(image_source_path, os.path.join(target_images_directory, image_file))
            print(f'Copied image file: {image_file}')
        else:
            print(f'Image file not found: {image_source_path}')


Copied label file: munster_000046_000019_leftImg8bit_foggy_beta_0.02.txt
Image file not found: /content/extracted_images/munster_000046_000019_leftImg8bit_foggy_beta_0.02.png
Copied label file: munster_000032_000019_foggy_beta_0.02.txt
Copied image file: munster_000032_000019_foggy_beta_0.02.png
Copied label file: munster_000046_000019_foggy_beta_0.02.txt
Copied image file: munster_000046_000019_foggy_beta_0.02.png
Copied label file: munster_000118_000019_foggy_beta_0.02.txt
Copied image file: munster_000118_000019_foggy_beta_0.02.png
Copied label file: lindau_000007_000019_leftImg8bit_foggy_beta_0.02.txt
Image file not found: /content/extracted_images/lindau_000007_000019_leftImg8bit_foggy_beta_0.02.png
Copied label file: munster_000010_000019_foggy_beta_0.02.txt
Copied image file: munster_000010_000019_foggy_beta_0.02.png
Copied label file: munster_000026_000019_foggy_beta_0.02.txt
Copied image file: munster_000026_000019_foggy_beta_0.02.png
Copied label file: munster_000000_000019_f

In [ ]:
 #Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
# Install the required dependencies
!pip install -r requirements.txt
%cd ..

Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 31.30 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
/content


In [ ]:
!pip install torch torchvision opencv-python matplotlib

In [ ]:
import shutil
import os
source_path = '/content/drive/MyDrive/foggy_IE643/train/labels'  # change accordingly
destination_path = '/content/input_foggy/train/labels'  # change accordingly
os.makedirs(destination_path, exist_ok=True)
filenames = os.listdir(source_path)
for filename in filenames:
    if filename.endswith('.txt'):
        source_file = os.path.join(source_path, filename)
        destination_file = os.path.join(destination_path, filename)
        shutil.copy(source_file, destination_file)

In [ ]:
import shutil
source_path = '/content/drive/MyDrive/foggy_IE643/train/images' #change accordingly
destination_path = '/content/input_foggy/train/images'  # change accordingly
os.makedirs(destination_path, exist_ok=True)
filenames = os.listdir(source_path)
for filename in filenames:
    if filename.endswith('.png'):
        source_file = os.path.join(source_path, filename)
        destination_file = os.path.join(destination_path, filename)
        shutil.copy(source_file, destination_file)

In [ ]:
# Path to the yolov5m.yaml file
yaml_path = "/content/yolov5/models/yolov5m.yaml"

# Open the file, read contents, and modify 'nc'
with open(yaml_path, 'r') as file:
    lines = file.readlines()

# Rewrite the file with updated nc
with open(yaml_path, 'w') as file:
    for line in lines:
        if line.strip().startswith('nc:'):
            file.write('nc: 5\n')  # Updateing to 5 classes from 80classes in yolo5vm.yaml
        else:
            file.write(line)

print("Updated yolov5m.yaml with nc=5.")

Updated yolov5m.yaml with nc=5.


In [ ]:
# Define paths and class names
train_images_path = '/content/input_foggy/train/images'  # Path to training images
train_labels_path = '/content/input_foggy/train/labels'  # Path to training labels
val_images_path = '/content/input_foggy/val/images'      # Path to validation images
val_labels_path = '/content/input_foggy/val/labels'      # Path to validation labels

# Define class names
class_names = ['bicycle', 'motorcycle', 'car', 'truck', 'bus']  # Adjust to your classes

# Create the data.yaml file
data_yaml_content = f"""
train: {train_images_path}
val: {val_images_path}

nc: {len(class_names)}  # Number of classes
names: {class_names}  # Class names
"""
os.makedirs('/content/working', exist_ok=True)
# Write to data.yaml file
with open('/content/working/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("data.yaml file created successfully!")


data.yaml file created successfully!


In [ ]:
classes_of_interest = {
    'bicycle': 0,
    'motorcycle': 1,
    'car': 2,
    'truck': 3,
    'bus': 4
}


In [ ]:
import os

# label files
label_dir = '/content/drive/MyDrive/foggy_IE643/train/labels'  # Change path if needed

class_counts = {label: 0 for label in classes_of_interest.keys()}

# Iterate over all label files
for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):  # Ensure it's a label file
        with open(os.path.join(label_dir, label_file), 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_id = int(float(line.split()[0]))  # class id extraction
                for class_name, id in classes_of_interest.items():
                    if class_id == id:
                        class_counts[class_name] += 1

# Print the number of images in each class
for class_name, count in class_counts.items():
    print(f"{class_name}: {count} instances")

total_instances = sum(class_counts.values())
num_classes = 5
#class_weights = {class_name: total_instances / (num_classes * count) for class_name, count in class_counts.items()}
#print(class_weights)
# use class_weights when no class is of zero number of classes in the dataset sued

bicycle: 241 instances
motorcycle: 63 instances
car: 441 instances
truck: 17 instances
bus: 33 instances


In [ ]:
import shutil
import numpy as np
import os
labels_here_dir = '/content/drive/MyDrive/foggy_IE643/train/labels'
images_here_dir = '/content/drive/MyDrive/foggy_IE643/train/images'
target_images_dir = '/content/working/filtered_train/images/nocar'
target_labels_dir = '/content/working/filtered_train/labels/nocar'
os.makedirs(target_images_dir, exist_ok=True)
os.makedirs(target_labels_dir, exist_ok=True)
count_removed = 0
car_class_id = 2
bicycle_class_id = 0
motorcycle_class_id = 1
truck_class_id = 3
bus_class_id = 4
for label_file in os.listdir(labels_here_dir):
    labels_path = os.path.join(labels_here_dir, label_file)
    with open(labels_path, 'r') as f:
        labels = f.readlines()

    # Count the number of class instances in the label file
    num_cars = sum(1 for label in labels if int(label.split()[0]) == car_class_id)
    num_bicycles = sum(1 for label in labels if int(label.split()[0]) == bicycle_class_id)
    num_buses = sum(1 for label in labels if int(label.split()[0]) == bus_class_id)
    num_trucks = sum(1 for label in labels if int(label.split()[0]) == truck_class_id)
    num_motorcycles = sum(1 for label in labels if int(label.split()[0]) == motorcycle_class_id)

    if  num_cars == 0:
        image_file = label_file.replace('.txt', '.png')
        image_path = os.path.join(images_here_dir, image_file)
        if os.path.exists(image_path):
            shutil.copy(image_path, target_images_dir)
            shutil.copy(labels_path, target_labels_dir)

            #os.remove(image_path)
            #os.remove(labels_path)
            count_removed += 1


# Print the count of images removed or copied
print(f"Removed/Copied {count_removed} images")

Removed 6 images and labels containing 8 or more cars.


In [ ]:
!pip install albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.7/680.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 kB 18.4 MB/s eta 0:00:00


In [ ]:
import albumentations as A
from keras.preprocessing.image import img_to_array, load_img

# directories
nocar_images_dir = '/content/drive/MyDrive/foggy_IE643/train/images'
augmented_nocar_images_dir = '/content/working/augmented_nocar1/images'
augmented_nocar_labels_dir = '/content/working/augmented_nocar1/labels'
os.makedirs(augmented_nocar_images_dir, exist_ok=True)
os.makedirs(augmented_nocar_labels_dir, exist_ok=True)
#augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.3),
    A.Blur(blur_limit=3, p=0.2),
    # Add other augmentations as needed
],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])
)

In [ ]:
import cv2
def load_bboxes(label_file_path):
    bboxes = []
    class_labels = []
    with open(label_file_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            bboxes.append([x_center, y_center, width, height])  # YOLO format
            class_labels.append(int(class_id))  # Store the class id separately
    return bboxes, class_labels

def save_bboxes(output_label_path, bboxes, class_labels):
    with open(output_label_path, 'w') as file:
        for bbox, class_id in zip(bboxes, class_labels):
            x_center, y_center, width, height = bbox
            file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

def augment_image(image_dir, label_file_dir, output_image_dir, output_label_dir, num_augments=20):
    # Load image and bounding boxes
    image_files = os.listdir(image_dir)
    for j, image_name in enumerate(image_files):
      label_file_path = os.path.join(label_file_dir, image_name.replace('.png', '.txt'))
      image_path = os.path.join(image_dir, image_name)
      image = cv2.imread(image_path)
      bboxes, class_labels = load_bboxes(label_file_path)

      for i in range(num_augments):
        # Apply augmentations
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_image = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_class_labels = augmented['class_labels']
        output_image_path = os.path.join(output_image_dir, f"{image_name.replace('.png', f'_aug_{i}.png')}")
        output_label_path = os.path.join(output_label_dir, f"{image_name.replace('.png', f'_aug_{i}.txt')}")
        # Save the augmented image
        cv2.imwrite(output_image_path, aug_image)

        # Save the corresponding labels with updated bounding boxes
        save_bboxes(output_label_path, aug_bboxes, aug_class_labels)

# usage
input_image_dir = '/content/working/filtered_train/images/1truck'
input_label_dir = '/content/working/filtered_train/labels/1truck'
output_image_dir = '/content/working/augmented_final/images'
output_label_dir = '/content/working/augmented_final/labels'


os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

augment_image(input_image_dir, input_label_dir, output_image_dir, output_label_dir, num_augments=10)

In [ ]:
source_dir = output_image_dir
destination_dir = '/content/drive/MyDrive/foggy_IE643/train/images'
os.makedirs(destination_dir, exist_ok=True)
label_files = os.listdir(source_dir)
for filename in label_files:
    if filename.endswith('.png'):
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, filename)
        shutil.move(source_path, destination_path)

In [ ]:
source_dir = output_label_dir
destination_dir = '/content/drive/MyDrive/foggy_IE643/train/labels'
os.makedirs(destination_dir, exist_ok=True)
label_files = os.listdir(source_dir)
num_files = 0
for filename in label_files:
    if filename.endswith('.txt'):
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, filename)
        shutil.move(source_path, destination_path)
        num_files += 1
print(num_files)

60


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!python train.py --img 640 --batch-size 8 --epochs 3 --data /content/working/data.yaml --cfg yolov5m.yaml --weights '/content/drive/MyDrive/yolov5_custom9/weights/best.pt' --name yolov5_foggy_finetune --project '/content/drive/MyDrive/yolov5_foggy_finetuning'


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
train: weights=/content/drive/MyDrive/yolov5_custom9/weights/best.pt, cfg=yolov5m.yaml, data=/content/working/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/yolov5_foggy_finetuning, name=yolov5_foggy_finetune, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity

In [ ]:
#/content/drive/MyDrive/yolov5_foggy_finetuning/yolov5_foggy_finetuning    /content/drive/MyDrive/yolov5_foggy_finetuning/yolov5_foggy_finetune2   /content/drive/MyDrive/yolov5_foggy_finetuning/yolov5_foggy_finetune

In [ ]:
!python train.py --img 640 --batch-size 8 --epochs 12 --data /content/working/data.yaml --cfg yolov5m.yaml --weights '/content/drive/MyDrive/yolov5_foggy_finetuning/yolov5_foggy_finetune2/weights/best.pt' --name yolov5_custom --project '/content/drive/MyDrive/yolov5_training'


train: weights=/content/drive/MyDrive/yolov5_foggy_finetuning/yolov5_foggy_finetune2/weights/best.pt, cfg=yolov5m.yaml, data=/content/working/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=12, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/yolov5_training, name=yolov5_custom, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.10.12 torch-2.5.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, wei

yolov5_custom10 is for for foggy finetuned forgot to change the name so here it is